In [ ]:
import os
import sys

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
import logging
from datetime import datetime as dt


now = dt.today()
logging.basicConfig(filename=f'{now}.debug.txt', level=logging.CRITICAL)

In [ ]:
username="fill_your_username"
password="fill_your_password"

In [ ]:
driver = webdriver.Firefox()
geektime_login = 'https://account.geekbang.org/login'
driver.get(geektime_login)

input_phone = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[1]/input")
input_pass = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[2]/input")

input_phone.send_keys(username)
input_pass.send_keys(password)

In [ ]:
# 登录 
button_submit = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/button")
button_submit.click()

In [ ]:
def logout():
    logout_node = None
    try:
        logout_node = driver.find_element_by_css_selector("div[class~='_1MA-AdXE_0'] a:last-child")
    except:
        try:
            logout_node = driver.find_element_by_css_selector("a.avatarwrap")
        except:
            pass
    
    if logout_node is not None:
        logout_node.click()
    else:
        account_node = None
        try:
            account_node = driver.find_element_by_css_selector("span[class~='iconfont']")
            account_node.click()
            logout_node = driver.find_element_by_css_selector("div[class~='_1MA-AdXE_0'] a:last-child")
        except Exception as e1:
            try:
                account_node = driver.find_element_by_css_selector("a.avatarwrap")
                account_node.click()
                logout_node = driver\
                            .find_element_by_css_selector("div[class~='profile-dropdown'] a:last-child")
            except Exception as e2: 
                logging.error("The account node cannot be found!")
                print(e1)
                print(e2)
        else:
            logout_node.click()
    try:
        WebDriverWait(driver, 10, 1).until(lambda d: d.find_element_by_css_selector("div#id"))
    except Exception as e:
        print(e, file=sys.stderr)
    finally:
        driver.quit()


In [ ]:
# 等待登陆成功 并进入我的基本信息页
WebDriverWait(driver, 20, 2).until(lambda x: x.find_elements_by_css_selector("a.navbar-item:nth-child(2)"))

In [ ]:
# click 极客时间
geektime_index = "https://time.geekbang.org/"
driver.get(geektime_index)

In [ ]:
# 我的课程列表的数据来自的是异步的AJAX请求 故在列表显示之前必须等待
WebDriverWait(driver, 10, 2).until(lambda x: x.find_element_by_xpath("/html/body/div[1]/div[1]/div[5]/div/a"))

In [ ]:
# click 关闭(modal)
driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[5]/div/a").click()

In [ ]:
# 点击 我的课程 无法让webdriver的活跃页跟随跳转到新打开的 我的课程 页,
# 直接请求 我的课程 URL 进行跳转
geektime_home = "https://time.geekbang.org/dashboard/course"
driver.get(geektime_home)

In [ ]:
# 我的课程列表的数据来自的是异步的AJAX请求 故在列表显示之前必须等待
course_list = None
try:
    course_list = WebDriverWait(driver, 20, 2).until(lambda x: x.find_elements_by_css_selector("button"))
    logging.info(course_list)
except Exception as e:
    print(e, file=sys.stderr)
    logout()

In [ ]:
course_home = driver.current_url
course_home

In [ ]:
# first course
course_list[0].click()

In [ ]:
try:
    WebDriverWait(driver, 20, 2).until(lambda x: len(x.window_handles) > 1)
except Exception as e:
    print(e, file=sys.stderr)
    logout()

Click on the button will open a new non-active tab, in other words, `driver.current_url` still remains 'https://time.geekbang.org/dashboard/course' while the course content is on the non-active tab.

In [ ]:
driver.current_url

In [ ]:
driver.window_handles

In [ ]:
## try to close current tab (old tab) to make the course page currentnt tab, but failed for some reason
# driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 'W')

## try to use CTRL + TAB to switch to the right tab, but failed again
#driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.TAB)

# finnaly this one works
driver.switch_to.window(driver.window_handles[1])

In [ ]:
# wait unit the chapter container is ready
WebDriverWait(driver, 20, 2)\
    .until(lambda x: x.find_element_by_xpath("//div[contains(@class, 'faLsNCZ2_0')]"))


In [ ]:
course_url = driver.current_url
course_url

In [ ]:
chapter_div_list = None
try:
    chapter_div_list = (WebDriverWait(driver, 20, 2)
                        .until(lambda d: d.find_elements_by_css_selector("div[class~='rMLmCpCA_0']")))
except Exception as e:
    print(e, file=sys.stderr)
    driver.refresh()
    (WebDriverWait(driver, 20, 2)
                        .until(lambda d: d.find_elements_by_css_selector("div[class~='rMLmCpCA_0']")))
        

In [ ]:
assert len(chapter_div_list) > 0, 'stop'

In [ ]:
chapter_title_list = []
for div in chapter_div_list:
    chapter_title_list.append(div.get_property("firstChild").get_property("firstChild")
                              .get_property("innerHTML"))

In [ ]:
chapter_title_list

In [ ]:
def format_path(p):
        # replace "/" with "," due to path format limitation
        p = p.replace(" ", "")
        p = p.replace('/', ',')
        # replace all non-ASCII code with ASCII code
        p = p.replace("丨", "|")
        p = p.replace("（", "(")
        p = p.replace("）",")")
        p = p.replace("：", ":")
        p = p.replace("？", "?")
        p = p.replace("，", ",")
        p = p.replace("“", "\"")
        p = p.replace("”","\"")
        return p


In [ ]:
chapter_title_list = list(map(format_path, chapter_title_list))
chapter_title_list

In [ ]:
course_title = driver.find_element_by_css_selector("div._382D6fck_0 > div.WRb7YQxC_0").text
course_title = format_path(course_title)

# create filesystem hierachy
if os.path.isdir(course_title) is False:
    os.mkdir(course_title)
    print(course_title)
    logging.info(course_title)
for chapter in chapter_title_list:
    p = os.path.join(course_title, chapter)
    print(p)
    logging.info(p)
    if os.path.isdir(p) is False:
        os.mkdir(p)


In [ ]:
# reset history and fold all chapter node to get ready for iteration
for chapter in chapter_div_list:
    logging.info(chapter.get_property("className"))
    if "_3kBNzF8S_0" in chapter.get_property("className"):
        chapter.get_property("firstChild").click()

In [ ]:
def save_article_to_html():
    course_title = (driver.find_element_by_css_selector("div._382D6fck_0 > div.WRb7YQxC_0")
                    .get_property("innerText"))
    
    current_article_div = driver.find_element_by_css_selector("div[class~='_2AkdpMxn_0']")
    
    current_chapter_div = (current_article_div
                            .get_property("parentElement")
                            .get_property("parentElement")
                            .get_property("parentElement"))
    current_chapter_name = (current_chapter_div
                            .get_property("firstChild")
                            .get_property("firstChild")
                            .get_property("innerText"))
    
    current_article_name = (current_article_div
                            .get_property("firstChild")
                            .get_property("innerText"))
    
    article_content_div = driver.find_element_by_css_selector("div._2c4hPkl9 > div:nth-child(1)")
    article_content_html = article_content_div.get_property("outerHTML")
    
    # write to file
    filename = current_article_name+'.html' 
    
    logging.info(course_title, current_chapter_name, current_article_name)
    output_path = os.path.join(format_path(course_title),
                               # format_path(current_chapter_name),
                               format_path(filename))
    logging.info(output_path)
    
    if os.path.isfile(output_path) is False:
        with open(output_path, 'w') as writer:
            writer.write(article_content_html)
        print("SUCCESS: " + output_path + " is saved.")
    else:
        print("WARNING: " + output_path + " has existed!")

In [ ]:
def save_all_article():
    n_chapter = len(chapter_div_list)

    for c in range(n_chapter):
        # first click the chapter div to unfold it
        chapter_div_list[c].get_property("firstChild").location_once_scrolled_into_view
        chapter_div_list[c].get_property("firstChild").click()

        article_node_list = (driver
                             .find_elements_by_css_selector(
                                 "div[class~='_3kBNzF8S_0'] div[class~='_1NHkz9xq_0']")
                            )
        n_article = len(article_node_list)

        for a in range(n_article):
            # click to go to the article
            article_node_list[a].location_once_scrolled_into_view
            article_node_list[a].click()

            # wait for content to be displayed
            try:
                WebDriverWait(driver, 20, 3).until(lambda x: x.find_element_by_css_selector("div._2c4hPkl9"))
            except Exception as e:
                logging.error(article_node_list[a].get_property("innerText")+" failed to save!")
            logging.info(driver.current_url)
            
            try:
                save_article_to_html()
            except Exception as e:
                logging.error(article_node_list[a].get_property("innerText")+" failed to save!")
                print(e, file=sys.stderr)

        current_chapter_div = driver.find_element_by_css_selector("div[class~='_3kBNzF8S_0']")
        current_chapter_div.get_property("firstChild").location_once_scrolled_into_view
        current_chapter_div.get_property("firstChild").click()


In [ ]:
save_all_article()

In [ ]:
logout()